In [1]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity

API_KEY = "db77747b3587407496f968ac23008831"
RESOURCE_ENDPOINT = "https://openai002.openai.azure.com/" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
#Note: The openai-python library support for Azure OpenAI is in preview.
openai.api_version = "2023-05-15"



### Data generation (Skip this step if data is already generated )

In [3]:
user_message = ""
def generate_airlines():
    user_message =f""" 
    Generate a list of 30 airline company names into a list format with company names separated by commas.
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The list of the airline company names is:

 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that knows a lot about the airline industry. You are helping a user find a list of airline company names."},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

airlines = generate_airlines()
airlines

'Air France, Alaska Airlines, American Airlines, British Airways, Cathay Pacific, Delta Air Lines, Emirates, Etihad Airways, Finnair, Frontier Airlines, Hawaiian Airlines, JetBlue Airways, KLM Royal Dutch Airlines, Lufthansa, Norwegian Air, Qantas Airways, Qatar Airways, Ryanair, Scandinavian Airlines, Singapore Airlines, Southwest Airlines, Spirit Airlines, Turkish Airlines, United Airlines, Virgin Atlantic, Virgin Australia, Vueling Airlines, Wizz Air, Xiamen Airlines, Air Canada, Air New Zealand.'

In [4]:
airlines = airlines.split(",")
airlines

['Air France',
 ' Alaska Airlines',
 ' American Airlines',
 ' British Airways',
 ' Cathay Pacific',
 ' Delta Air Lines',
 ' Emirates',
 ' Etihad Airways',
 ' Finnair',
 ' Frontier Airlines',
 ' Hawaiian Airlines',
 ' JetBlue Airways',
 ' KLM Royal Dutch Airlines',
 ' Lufthansa',
 ' Norwegian Air',
 ' Qantas Airways',
 ' Qatar Airways',
 ' Ryanair',
 ' Scandinavian Airlines',
 ' Singapore Airlines',
 ' Southwest Airlines',
 ' Spirit Airlines',
 ' Turkish Airlines',
 ' United Airlines',
 ' Virgin Atlantic',
 ' Virgin Australia',
 ' Vueling Airlines',
 ' Wizz Air',
 ' Xiamen Airlines',
 ' Air Canada',
 ' Air New Zealand.']

In [7]:
user_message = ""
def generate_polity_types():
    user_message =f""" 
    Generate a list of policy types for airline industry separated by commas that covers everything from luggage policy to pet policy to food policy to ticket change policy.
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The list of the policy names is:

 
"""

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that knows a lot about the airline industry. You are helping a user find a list of airline company names."},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            

policty_types = generate_polity_types()
policty_types = policty_types.split(",")

policty_types

['luggage policy',
 ' carry-on policy',
 ' checked baggage policy',
 ' overweight baggage policy',
 ' oversized baggage policy',
 ' pet policy',
 ' in-flight food policy',
 ' special meal policy',
 ' beverage policy',
 ' seating policy',
 ' boarding policy',
 ' flight change policy',
 ' cancellation policy',
 ' travel insurance policy.']

In [19]:
import time
def generate_policy(airline, policy_type):
    user_message =f""" 
    generate detail content for {policy_type} for {airline} company. 
    Just output the data, no need for any comments or explanations as it will break downstream applications.
    The detail content is:
 
"""
    i=0
    while i<5:
        try:
            response = openai.ChatCompletion.create(
                engine="gpt-35-turbo", # engine = "deployment_name".
                messages=[
                    {"role": "system", "content": f"You are a helpful AI assistant at {airline} that knows a lot about the airline industry and the company."},
                    {"role": "user", "content":user_message },
                ]
            )
            return response['choices'][0]['message']['content']
        except:
            i+=1
            time.sleep(5)
            


In [10]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.models import Vector 
import openai
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)  

AZURE_OPENAI_ENDPOINT="https://openai002.openai.azure.com/"
AZURE_OPENAI_API_KEY="db77747b3587407496f968ac23008831"
AZURE_OPENAI_EMB_DEPLOYMENT="text-embedding-ada-002"
AZURE_SEARCH_SERVICE_ENDPOINT="https://cogsearch001.search.windows.net"
AZURE_SEARCH_ADMIN_KEY="2HwNw2JAmUjp8lYFfiiIMeIFTPye8rHHq9ZYu3hc9YAzSeA9Vl5L"

credential = AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
index_name="airline-policy"
# Create a search index
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="airline", type=SearchFieldDataType.String,
                    filterable=True),
    SearchableField(name="active", type=SearchFieldDataType.String,
                    filterable=True),

    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        HnswVectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 500,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="airline")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')




azcs_search_client = SearchClient(AZURE_SEARCH_SERVICE_ENDPOINT, index_name =index_name , credential=credential)
def generate_embeddings(text):
    openai.api_version = "2023-05-15"
    response = openai.Embedding.create(
        input=text, engine=AZURE_OPENAI_EMB_DEPLOYMENT)
    embeddings = response['data'][0]['embedding']
    return embeddings

def add_or_update(document):
    azcs_search_client.merge_or_upload_documents(documents = document)



 airline-policy created


In [20]:
import uuid
policy_list = []
for airline in airlines[:3]:
    for policty_type in policty_types:
        policy = generate_policy(airline, policty_type)
        policy_type_emb = generate_embeddings(policty_type)
        policy_emb = generate_embeddings(policy)
        policy_document= {"id" : str(uuid.uuid4()), "airline": airline, "policy_type": policty_type, "policy": policy,"titleVector":policy_type_emb, "contentVector":policy_emb, "active": "True"}
        policy_list.append(policy_document)
        

emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine text-embedding-ada-002
emb_engine tex

In [18]:
policy_list[0]

{'id': '9d611616-5788-4854-821d-eea8d3773790',
 'airline': 'Air France',
 'policy_type': 'luggage policy',
 'policy': "Thank you for contacting ADP customer support. As an ADP customer support agent, I am unable to generate content for a different company. However, I can provide you with the information you need regarding luggage policy for Air France.\n\nAir France follows the standardized baggage policy for international flights. The airline has specific rules and restrictions for cabin and checked-in luggage.\n\nCabin luggage policy:\n\nAir France allows one piece of cabin luggage, which includes handbags, laptop cases, and briefcases. The weight limit for cabin luggage is 12 kg, and the dimension should not exceed the maximum size of 55 x 35 x 25 cm.\n\nNote: Passengers travelling in premium economy, business class, and La Première are allowed to carry two cabin luggage items.\n\nChecked baggage policy:\n\nAir France allows a baggage weight limit of 23 kg for standard economy class

In [ ]:
policy_list = policy_list

In [16]:
add_or_update(policy_list)


HttpResponseError: () The request is invalid. Details: Cannot convert the literal 'True' to the expected type 'Edm.String'.
Code: 
Message: The request is invalid. Details: Cannot convert the literal 'True' to the expected type 'Edm.String'.

In [17]:
import os
folder_path ="data"
os.makedirs(folder_path, exist_ok=True)
i=0
articles ={}
file_name="policy_vector.json"
with open(os.path.join(folder_path,file_name), 'w') as file:
        file.write(policy_list)




TypeError: write() argument must be str, not list

### Generate embeddings

In [ ]:
openai.api_version = "2022-12-01"

articles_emb = {article[0]:get_embedding(article[1], engine = 'text-embedding-ada-002')for article in list(articles.items())}

### Persist data

In [ ]:
import json

with open(os.path.join(folder_path,"articles_emb.json"), "w") as fp:
    json.dump(articles_emb,fp) 

with open(os.path.join(folder_path,"articles.json"), "w") as fp:
    json.dump(articles,fp) 


## Assistant Design

### Load data

In [ ]:
import json
import os
folder_path ="../../data/agent_assistant"
with open(os.path.join(folder_path, "articles_emb.json"), "r") as file:
    articles_emb = json.load(file)
with open(os.path.join(folder_path, "articles.json"), "r") as file:
    articles = json.load(file)

### 1. Tool to generate conversation

In [ ]:
openai.api_version = "2023-05-15"

def generate_conversation(streaming =False):
    user_message =f""" 
    Generate a phone conversation between an customer's employee and an ADP support agent in the area of HR and Payroll. The employee has question and the support agent tried to identity the problems and 
    and take time to use different search tool to come up with answer to the employee.
    Your response:
 
"""
    system_message = """
    You are a customer support agent for ADP company. 

    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ],
        stream=streaming
    )
    return response
            


### 2. Tool to extract problems from conversation. 

In [ ]:
openai.api_version = "2023-05-15"

user_message = ""
def extract_problems(conversation):
    user_message =f""" 
    Follow this on going conversation below and extract problems that each party may need help with and formulate the search query to the knowledge base search tool.
    <<conversattion>>
    {conversation}
    <<conversattion>>
    Output your response in JSON format {{"problem":"summary of problem", "search_query":"content of search query"}}
    There can be more than 1 problem(s)
    Output just JSON, nothing else.
    Your response:
 
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    You are given access to knowledge base search tool to find knowledge needed to find answer to questions. 

    """

    response = openai.ChatCompletion.create(
        timeout=100,
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content']
            
conversation=generate_conversation()['choices'][0]['message']['content']
problems=extract_problems(conversation)
problems

### 3. Tool to find article given a problem

##### Brute Force method

In [ ]:
import numpy as np  
openai.api_version = "2022-12-01"

def find_article(question, topk=5):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    max_similarity = float('-inf')  
    best_file_name = None  
      
    # Compute cosine similarity between input vector and each label vector
    cosine_list=[]  
    for file_name, vector in articles_emb.items():  
        cosine_sim = cosine_similarity(input_vector, vector) 
        cosine_list.append((file_name,cosine_sim ))
    cosine_list.sort(key=lambda x:x[1],reverse=True)
    cosine_list= cosine_list[:topk]
    print(cosine_list)
    best_file_names = [file_name[0] for file_name in cosine_list]
    contents = [articles[best_file_name] for best_file_name in best_file_names]
    return best_file_names, contents
question = "How do I update employee information"
find_article(question)

#### Vector Searh Lib (Faiss) method

In [ ]:
import numpy as np  
import faiss

openai.api_version = "2022-12-01"
#Get the array of embeddings for all articles
file_names =[]
emb_vectors = []
for file_name, vector in articles_emb.items():  
    file_names.append(file_name)
    emb_vectors.append(vector)
emb_vectors = np.float32(emb_vectors)
faiss.normalize_L2(emb_vectors)

index = faiss.IndexFlatIP(len(vector)) 
index.add(emb_vectors)


def find_article_emb_vec(question, topk=3):  
    output_articles=[]
    output_contents =[]
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    input_vector = get_embedding(question, engine = 'text-embedding-ada-002')
    input_vector = np.float32([input_vector])
    faiss.normalize_L2(input_vector)
    d,i = index.search(input_vector, k=topk)
    print(d)
    for idx in i[0]:
        output_articles.append(file_names[idx])
        output_contents.append(articles[file_names[idx]])

    return output_articles, output_contents
question = "How do I update employee information"
find_article_emb_vec(question,5)

### 4. Tool to answer question from the given problem

In [ ]:
openai.api_version = "2023-03-15-preview"

def answer_assist(problem, search_query):

    articles, contents = find_article_emb_vec(search_query,2)
    articles_contents=""
    for article, content in zip(articles, contents):
        articles_contents += f""" 
        article:{article}
        content: {content}
    """
    articles_contents = f"""
    <<knowledge articles>>
    {articles_contents}
    <<knowledge articles>>
    """
    user_message =f""" 
    problem:{problem}
    {articles_contents}
    Your response:
"""
    system_message = """
    You are a senior customer support agent for ADP company. You listen to the conversation between an agent and a customer and assist the agent to resolve the problem.
    Given the question or problem statement and the knowledge article you have, give the answer.
    Rely solely to the guidance from the article.If the knowlege article is not relavant to the question, say you don't know. Do not make up your answer. 
    Cite the name of the knowledge article as source for your answer.
    Format:
    Answer: your answer to the question
    Sources: [source1, source2]
    """

    response = openai.ChatCompletion.create(
        engine="gpt-35-turbo", # engine = "deployment_name".
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content":user_message },
        ]
    )
    return response['choices'][0]['message']['content'], articles_contents

question = "When do I receive my W2 form?"
answer, articles_contents = answer_assist(question,question)
print(answer)
print("---------------content-----------------")

print(articles_contents)

### 5. Put tools all together

In [ ]:
def recommend(conversation):
    i=0
    while (i<5): #handle wrong format output
        problems=extract_problems(conversation)
        try:
            problems=json.loads(problems)
            print("problems", problems)
            for problem in problems:
                answer, articles_contents = answer_assist(problem['problem'],problem['search_query'])
                print(answer)
                print("---------------content-----------------")

                print(articles_contents)
            break

        except Exception as e:
            print(e)
            print("problem parsing json, problems string is ", problems)
            i+=1

        



### One time conversation 

In [ ]:
conversation =generate_conversation()['choices'][0]['message']['content']
print(f"Conversation {conversation}")
recommend(conversation)        

    

### Simulate a running conversation 

In [ ]:
import concurrent
openai.api_version = "2023-05-15"
import time
conversation_pause_duration=2
agent_assist_freq=5
conversation_buffer =[]
def stream_conversation(conversation_buffer, pause_duration=5):
    responses = generate_conversation(True)
    conversation_counter =0
    old_conversation_counter =0
    for response in responses:
        content = response['choices'][0]["delta"].get("content","")
        conversation_buffer.append(content) 
        if "\n"  not in content:
            print(content, end="")
        else:
            conversation_counter+=1
            if conversation_counter > old_conversation_counter+2:
                conversation = "".join(conversation_buffer)
                print("starting recommendation")
                print("conversation: ", conversation)
                recommend(conversation)
                print("ending recommendation")
                old_conversation_counter=conversation_counter



        
stream_conversation(conversation_buffer,conversation_pause_duration)